# Model attention analysis
We assume that the social-token model has learned to "pay attention" to different aspects of input text based on different social groups' expectations.

Let's test this by analyzing the attention distribution for the social-token model for different inputs!

In [1]:
## load test data
import torch
test_data = torch.load('../../data/reddit_data/combined_data_test_data.pt')
data_cols = ['source_text', 'target_text', 'article_id', 'id', 'author', 'reader_token_str', 'source_ids', 'source_ids_reader_token', 'attention_mask']
test_data = test_data.data.to_pandas().loc[:, data_cols]
display(test_data.head())

,source_text,target_text,article_id,id,author,reader_token_str,source_ids,source_ids_reader_token,attention_mask
0,I live in Florida I haven't filed my taxes in ...,"Before those years, did you have a good histor...",b49v6a,ej5dvzk,these-things-happen,UNK,"[0, 100, 697, 11, 1261, 38, 2220, 75, 1658, 12...","[0, 100, 697, 11, 1261, 38, 2220, 75, 1658, 12...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,I just came off the phone with my very enraged...,Is there a way to celebrate like on the 23rd o...,dwavsu,f7hp94x,smithjojo99,UNK,"[0, 100, 95, 376, 160, 5, 1028, 19, 127, 182, ...","[0, 100, 95, 376, 160, 5, 1028, 19, 127, 182, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,This happened in college a few years back and ...,Did you normally give back the change?,9mfdhu,e7eccty,LearnedButt,<EXPERT_PCT_0_AUTHOR>,"[0, 713, 1102, 11, 1564, 10, 367, 107, 124, 8,...","[0, 1102, 11, 1564, 10, 367, 107, 124, 8, 4596...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,I recently went on a short 4-day holiday and o...,I don't think you are an asshole but is there ...,88k7t3,dwlbnpc,justdeadstars,UNK,"[0, 100, 682, 439, 15, 10, 765, 204, 12, 1208,...","[0, 100, 682, 439, 15, 10, 765, 204, 12, 1208,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,"As far as I know, any child born abroad to a l...",I'm guessing they didn't follow the procedure ...,cy5vc6,eypwupx,UsuallySunny,<RESPONSE_TIME_0_AUTHOR>,"[0, 1620, 444, 25, 38, 216, 6, 143, 920, 2421,...","[0, 444, 25, 38, 216, 6, 143, 920, 2421, 5358,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [2]:
## identify posts w/ questions from different groups 
reader_group_token_lookup = {
    'location_region' : ['<US_AUTHOR>', '<NONUS_AUTHOR>'],
    'expert_pct_bin' : ['<EXPERT_PCT_0_AUTHOR>', '<EXPERT_PCT_1_AUTHOR>'],
    'relative_time_bin' : ['<RESPONSE_TIME_0_AUTHOR>', '<RESPONSE_TIME_1_AUTHOR>'],
}
reader_token_group_lookup = {
    v : k for k, vs in reader_group_token_lookup.items() for v in vs
}
test_data = test_data.assign(**{
    'reader_group' : test_data.loc[:, 'reader_token_str'].apply(reader_token_group_lookup.get)
})
reader_group_test_data = test_data.dropna(subset=['reader_group'])
print(reader_group_test_data.loc[:, 'reader_group'].value_counts())

expert_pct_bin       14324
relative_time_bin    14324
location_region       2648
Name: reader_group, dtype: int64


In [3]:
import pandas as pd
from tqdm import tqdm
id_with_all_groups = []
post_id_var = 'article_id'
reader_group_var = 'reader_group'
for id_i, data_i in tqdm(reader_group_test_data.groupby(post_id_var)):
    for group_j, data_j in data_i.groupby(reader_group_var):
        if(data_j.loc[:, 'reader_token_str'].nunique()==2):
            id_with_all_groups.append([id_i, group_j])
id_with_all_groups = pd.DataFrame(id_with_all_groups, columns=[post_id_var, reader_group_var])
paired_reader_group_test_data = pd.merge(reader_group_test_data, id_with_all_groups, on=[post_id_var, reader_group_var], how='inner')
# sort values to make lookup less annoying
paired_reader_group_test_data.sort_values([post_id_var, reader_group_var], inplace=True, ascending=True)
display(paired_reader_group_test_data.loc[:, reader_group_var].value_counts())

100%|██████████| 11272/11272 [00:09<00:00, 1169.08it/s]


relative_time_bin    2681
expert_pct_bin       2508
location_region       280
Name: reader_group, dtype: int64

## Compute attention distributions
Let's compute the attention distributions from the social-token model.

In [4]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [5]:
## load model lol
from model_helpers import load_model
model_cache_dir = '../../data/model_cache/'
model_file = '../../data/reddit_data/author_text_data/question_generation_model/checkpoint-129000/pytorch_model.bin'
model_type = 'bart_author_token'
data_dir = '../../data/reddit_data/'
model, tokenizer = load_model(model_cache_dir, model_file, model_type, data_dir)
model = model.eval()

In [6]:
# test input
import torch
import re
import numpy as np
test_input_data = paired_reader_group_test_data.iloc[:2]
test_output_attn_data = []
test_output_reader_tokens = []
for idx_i, data_i in test_input_data.iterrows():
    test_output_attn_data_i = []
    test_output_reader_tokens.append(data_i.loc['reader_token_str'])
#     print(f'group = {data_i.loc["reader_token_str"]}')
# test_input_data = paired_reader_group_test_data.iloc[0]
    test_input_ids = torch.LongTensor(data_i.loc['source_ids_reader_token']).unsqueeze(0)
    test_attention_mask = torch.LongTensor(data_i.loc['attention_mask']).unsqueeze(0)
    with torch.no_grad():
        test_output = model(input_ids=test_input_ids.to(model.device), 
                            attention_mask=test_attention_mask.to(model.device), 
                            output_attentions=True)
    # print(test_output.encoder_attentions[0].shape)
    ## cannot use bertviz because of long sequence
    # use first layer of attention?
    # compute mean over all heads, then mean over all tokens
    test_output_attn = np.array(test_output.encoder_attentions[0].mean(dim=1).mean(dim=1).squeeze(0).cpu())
    # show raw values
    test_input_tokens = tokenizer.convert_ids_to_tokens(test_input_ids[0], skip_special_tokens=True) # drop PAD tokens
    test_input_tokens = list(map(lambda x: re.sub('^[Ġ]', '', x), test_input_tokens))
    for token_i, attn_i in zip(test_input_tokens, test_output_attn):
        test_output_attn_data_i.append([token_i, attn_i])
#         print(f'{attn_i:.3f} ({token_i})')
    test_output_attn_data_i = pd.DataFrame(test_output_attn_data_i, columns=[f'token_{data_i.loc["reader_token_str"]}', f'attn_{data_i.loc["reader_token_str"]}'])
    test_output_attn_data.append(test_output_attn_data_i)
test_output_attn_data = pd.concat(test_output_attn_data, axis=1)
# compute diff
test_output_reader_token_1, test_output_reader_token_2 = test_output_reader_tokens
test_output_attn_data = test_output_attn_data.assign(**{
    'attn_diff' : test_output_attn_data.loc[:, f'attn_{test_output_reader_token_1}'] / test_output_attn_data.loc[:, f'attn_{test_output_reader_token_2}']
})
# get rank by attn diff
test_output_attn_data = test_output_attn_data.assign(**{
    'attn_diff_rank' : test_output_attn_data.loc[:, 'attn_diff'].argsort()
})

In [7]:
pd.set_option('display.max_rows', 500)
display(test_output_attn_data)

,token_<NONUS_AUTHOR>,attn_<NONUS_AUTHOR>,token_<US_AUTHOR>,attn_<US_AUTHOR>,attn_diff,attn_diff_rank
0,gin,0.065053,gin,0.065047,1.000087,153
1,ally,0.004280,ally,0.004258,1.005267,218
2,I,0.004908,I,0.004906,1.000445,89
3,was,0.001142,was,0.001140,1.002111,64
4,quoted,0.002210,quoted,0.002211,0.999664,47
5,a,0.004749,a,0.004741,1.001519,93
6,salary,0.002113,salary,0.002114,0.999807,219
7,of,0.003699,of,0.003690,1.002454,50
8,97,0.002850,97,0.002845,1.001557,210
9,k,0.015699,k,0.015628,1.004534,159


For every post, let's get the attention distributions for each pair of authors in group G1 and G2, compute the KL divergence between A_{G1} and A_{G2}, get posts w/ high divergence.

In [8]:
import numpy as np
def compute_attn_distribution(input_data, model, layer=0):
    input_ids = torch.LongTensor(input_data.loc['source_ids_reader_token']).unsqueeze(0)
    attention_mask = torch.LongTensor(input_data.loc['attention_mask']).unsqueeze(0)
    with torch.no_grad():
        model_output = model(input_ids=input_ids.to(model.device), 
                             attention_mask=attention_mask.to(model.device), 
                             output_attentions=True)
    model_output_attn = np.array(model_output.encoder_attentions[layer].mean(dim=1).mean(dim=1).squeeze(0).cpu())
    return model_output_attn

In [10]:
from tqdm import tqdm
tqdm.pandas()
attn_layer = 0
paired_reader_group_test_data = paired_reader_group_test_data.assign(**{
    'input_attention' : paired_reader_group_test_data.progress_apply(lambda x: compute_attn_distribution(x, model, layer=attn_layer), axis=1)
})

100%|██████████| 5469/5469 [04:26<00:00, 20.49it/s]


In [11]:
## save, reload to release GPU memory ;_;
# save arrays as lists for easier loading
list_cols = ['input_attention', 'source_ids', 'source_ids_reader_token', 'attention_mask']
paired_reader_group_test_data = paired_reader_group_test_data.assign(**{
    c : paired_reader_group_test_data.loc[:, c].apply(list)
    for c in list_cols
})
paired_reader_group_test_data.to_csv('paired_test_data_tmp.gz', sep='\t', compression='gzip', index=False)

In [1]:
## reload
import pandas as pd
from ast import literal_eval
import sys
if('..' not in sys.path):
    sys.path.append('..')
from data_processing.data_helpers import str2array
list_cols = ['source_ids', 'source_ids_reader_token', 'attention_mask', 'input_attention']
converters = {
    c : literal_eval
    for c in list_cols
}
paired_reader_group_test_data = pd.read_csv('paired_test_data_tmp.gz', sep='\t', converters=converters)
# limit attention to actual input length
import numpy as np
PAD_TOKEN_ID = 1
def clip_attn(attention, input_ids):
    attn_end_idx = len(input_ids)
    pad_token_idx = np.where(np.array(input_ids)==PAD_TOKEN_ID)[0]
    if(len(pad_token_idx) > 0):
        attn_end_idx = pad_token_idx.min()
    # remove <START> and <READER_TOKEN>
    attention = attention[1:(attn_end_idx-1)]
    return attention
paired_reader_group_test_data = paired_reader_group_test_data.assign(**{
    'input_attention' : paired_reader_group_test_data.apply(lambda x: clip_attn(x.loc['input_attention'], x.loc['source_ids_reader_token']), axis=1)
})

In [2]:
## get tokens
import torch
import re
tokenizer = torch.load('../../data/reddit_data/author_text_data/BART_tokenizer.pt')
paired_reader_group_test_data = paired_reader_group_test_data.assign(**{
    'input_tokens' : paired_reader_group_test_data.loc[:, 'source_ids_reader_token'].apply(lambda x: list(map(lambda y: re.sub('^[Ġ]', '', y), tokenizer.convert_ids_to_tokens(x, skip_special_tokens=True))))
})

In [3]:
from scipy.special import kl_div
def compute_attn_divergence(data, group_var='reader_token_str', attn_var='input_attention'):
    """
    Compute divergence between groups,
    also reshape data.
    """
    reader_groups = data.loc[:, group_var].unique()
    group_1, group_2 = reader_groups
    data_1 = data[data.loc[:, group_var]==group_1].iloc[0, :]
    data_2 = data[data.loc[:, group_var]==group_2].iloc[0, :]
    div_1_2 = kl_div(data_1.loc[attn_var], data_2.loc[attn_var]).sum()
    # tokens_1 | group_1 | attn_1 | tokens_2 | group_2 | attn_2 | divergence
    div_data_cols = ['input_tokens', group_var, attn_var]
    div_data = pd.concat([
        data_1.loc[div_data_cols].rename({c : f'{c}_1' for c in div_data_cols}),
        data_2.loc[div_data_cols].rename({c : f'{c}_2' for c in div_data_cols}),
    ])
    div_data.loc['group_1'] = group_1
    div_data.loc['group_2'] = group_2
    div_data.loc['attn_div'] = div_1_2
    # add "normal" input tokens
    div_data.loc['input_tokens'] = div_data.loc['input_tokens_1']
    return div_data

In [4]:
id_var = 'article_id'
group_var = 'reader_group'
attn_div_data = []
for (id_i, group_i), data_i in paired_reader_group_test_data.groupby([id_var, group_var]):
    div_data_i = compute_attn_divergence(data_i)
    div_data_i.loc[group_var] = group_i
    div_data_i.loc[id_var] = id_i
    attn_div_data.append(div_data_i)
attn_div_data = pd.concat(attn_div_data, axis=1).transpose()
# also get raw input text for easier viewing
attn_div_data = attn_div_data.assign(**{
    'source_text' : attn_div_data.loc[:, 'input_tokens_1'].apply(lambda x: ' '.join(x[:-1]))
})

In [7]:
pd.set_option('display.max_colwidth', 1000)
display_cols = ['group_1', 'group_2', 'source_text', 'input_attention_1', 'input_attention_2', 'attn_div']
for group_i, data_i in attn_div_data.groupby('reader_group'):
    data_i.sort_values('attn_div', ascending=False, inplace=True)
    print(f'group={group_i}')
    display(data_i.loc[:, display_cols].head(10))

group=expert_pct_bin


,group_1,group_2,source_text,input_attention_1,input_attention_2,attn_div
593,<EXPERT_PCT_0_AUTHOR>,<EXPERT_PCT_1_AUTHOR>,"I can 't deal with my parents anymore but unfortunately , you can 't file as an independent until you 're 24 and I 'm 18 and a college junior . If I wanted to marry someone literally just for F AS FA would there be any actual problems ? I guess that it could be considered unethical but I like 80 % of my college is already paid for through Bright Fut ures and the FL State grant it 's just that I need my parents to sign for F AS FA . If I did this and then divorced the person after I get into grad school would there be a problem ?","[0.0019550372, 0.0034003402, 0.0029003527, 0.0054153083, 0.004307159, 0.00281363, 0.0038389084, 0.0063671772, 0.0024802692, 0.007037138, 0.0068509467, 0.0028487595, 0.0035997755, 0.003296809, 0.00464324, 0.0044449028, 0.005276286, 0.0053407843, 0.008855231, 0.0026119403, 0.004809672, 0.0048482968, 0.0036635213, 0.003490619, 0.0049295947, 0.00341277, 0.0042533493, 0.0048487317, 0.0061361017, 0.00874549, 0.008004145, 0.004400553, 0.005007634, 0.0058414387, 0.006541002, 0.008584058, 0.0032003466, 0.0060927304, 0.003383433, 0.0055569955, 0.0041262396, 0.0048697446, 0.011329683, 0.0056323493, 0.004617183, 0.004545777, 0.004391916, 0.005480661, 0.005399121, 0.0047496576, 0.0032361194, 0.005959294, 0.0029313539, 0.0045742234, 0.0042570992, 0.0034903958, 0.003928134, 0.012677498, 0.004141916, 0.0029986172, 0.0028660162, 0.0040249676, 0.003439186, 0.0057802624, 0.003563331, 0.0054704086, 0.0040623806, 0.005761546, 0.0037147386, 0.0056445766, 0.00694219, 0.028202748, 0.08005425, 0.0077761314...","[0.0019839886, 0.003400834, 0.0029041516, 0.005446761, 0.0043042535, 0.0028382838, 0.0038570964, 0.006414447, 0.002487842, 0.0071890643, 0.0068841763, 0.002858169, 0.0035782794, 0.003299824, 0.004665374, 0.004459153, 0.0052765985, 0.0053669116, 0.008986492, 0.0026174816, 0.004815117, 0.0048250575, 0.0036947033, 0.0035237262, 0.004940683, 0.003429441, 0.0043073827, 0.004853717, 0.0061514177, 0.008956302, 0.008044838, 0.004403361, 0.005035302, 0.005847381, 0.0065551884, 0.008673008, 0.0032036733, 0.0061102, 0.0033855336, 0.0055642, 0.0041401316, 0.004951226, 0.011430013, 0.005636352, 0.0046337424, 0.0045603383, 0.0043936106, 0.0054249354, 0.0054173707, 0.0047491286, 0.0032638349, 0.0059144516, 0.0029586675, 0.004592142, 0.004262815, 0.0034814808, 0.00393535, 0.012800914, 0.0041491883, 0.0030242342, 0.002865455, 0.0040307934, 0.0034505688, 0.0057677366, 0.0035866199, 0.005492348, 0.004067849, 0.0057834405, 0.0037244912, 0.005659535, 0.0069705094, 0.029732106, 0.09059796, 0.007933917, ...",0.001248
186,<EXPERT_PCT_0_AUTHOR>,<EXPERT_PCT_1_AUTHOR>,"wife had an old 401 k with about $ 15 , 000 in it from a job she left several years ago . The 401 k was with Wells Fargo . 1 month ago I noticed Wells had been charging about $ 20 annually in administration fees , which annoyed me . So we moved all that money into a Roll over IRA with Schw ab . All those dollars are now invested in lower cost investments , and with a much more trustworthy company ( Sch w ab ). Victory ! & amp ; # x 200 B ; However , my wife called Wells today to give them the final middle finger and close the account , but Wells told her "" your old employer needs to do that , or you 'll need to wait several years for the account to close ."" & amp ; # x 200 B ; My BS detector is going off . Is this true ? & amp ; # x 200 B ; If no , then any recommendations ? & amp ; # x 200 B ; If yes , then I have other concerns . If we try to contact her old employer ( a mega corporation ), I 'm concerned it 'll take tons of time because I have no idea who to contact . An...","[0.0015061318, 0.0019442467, 0.0017070821, 0.0018804994, 0.004461615, 0.0014316454, 0.0018998552, 0.00203843, 0.0013407624, 0.0017580276, 0.0024988311, 0.0024003964, 0.0027846254, 0.001687012, 0.0025495565, 0.001655834, 0.0021109546, 0.0013730344, 0.0023244428, 0.0025926256, 0.00186

group=location_region


,group_1,group_2,source_text,input_attention_1,input_attention_2,attn_div
866,<US_AUTHOR>,<NONUS_AUTHOR>,"found out I was pregnant with my daughter at 19 years old , had her at 20 . When I told my parents they said they ' ld be happy for me but they didn 't want to pay for anything , have us live with them or babys it all the time , basically wanting to treat me they same they would if I had kids older and not as a teenager still going to university . I was already living independently , both my boyfriend and I worked and we live in a country with free / che ap education and healthcare . In our culture families are close and grandparents are very involved in their grandchildren 's lives , so I thought they 'd still be emotionally supportive . During my pregnancy I asked my parents and / or sister for help a few times : driving me to the hospital when I started contracting 3 months early , picking me up for a family event when I wasn 't allowed to drive , helping me prepare meals to put in the freezer , and staying with them for a weekend when I was due any day and my boyfriend couldn '...","[0.0013039489, 0.00096310274, 0.0006313581, 0.0012142446, 0.0025696089, 0.001057381, 0.0008620015, 0.0017641602, 0.0011955465, 0.0017078256, 0.0014395022, 0.0014191829, 0.0017313239, 0.0013299277, 0.0010500457, 0.0012782912, 0.001412692, 0.0017980749, 0.002576684, 0.0007874131, 0.0013152767, 0.00092043914, 0.0015505322, 0.0013525093, 0.0012358829, 0.001030858, 0.0014599566, 0.0023566731, 0.0010495005, 0.0019136496, 0.0015009172, 0.00096421305, 0.0011559825, 0.0010815641, 0.0010255815, 0.00093445927, 0.0009811214, 0.001363639, 0.0010731247, 0.0013115027, 0.0010701965, 0.0016765861, 0.0008354089, 0.0012391317, 0.0012842093, 0.0011331545, 0.0011627044, 0.0011586855, 0.003079529, 0.0010678786, 0.00091272546, 0.0010535959, 0.0010230418, 0.0018528296, 0.0013920784, 0.0014050159, 0.0013016324, 0.001992026, 0.0008877266, 0.000981815, 0.0014839006, 0.0009269305, 0.0009729508, 0.0009080799, 0.00069214415, 0.0012699731, 0.0010454087, 0.0017387695, 0.001228067, 0.000809471, 0.0010721893, 0.001...","[0.001312359, 0.0009643857, 0.00063396816, 0.0012160039, 0.0025875831, 0.0010577614, 0.00086472713, 0.0017701316, 0.0011982028, 0.0017109739, 0.001444042, 0.0014212054, 0.0017371997, 0.0013312325, 0.001051511, 0.001280771, 0.001415486, 0.0018149287, 0.0025902987, 0.00079052814, 0.0013179304, 0.0009240996, 0.0015539343, 0.0013542715, 0.0012400122, 0.0010323372, 0.0014638978, 0.0023957067, 0.0010508222, 0.0019241389, 0.0015023225, 0.000967214, 0.0011576604, 0.0010838113, 0.001026338, 0.00093544275, 0.0009819727, 0.001365149, 0.0010735156, 0.0013109007, 0.0010706221, 0.0016805453, 0.0008357056, 0.0012431046, 0.0012864866, 0.0011342318, 0.0011653352, 0.0011560065, 0.003085543, 0.0010739141, 0.0009131969, 0.0010541517, 0.001024304, 0.0018564113, 0.001395174, 0.0014081295, 0.0013028705, 0.0020063352, 0.00089014845, 0.0009831407, 0.0014860968, 0.00092794956, 0.00097356865, 0.00090824, 0.00069521373, 0.0012727623, 0.0010464969, 0.0017466007, 0.0012351503, 0.0008098242, 0.0010753013, 0.0015...",0.000362
25,<NONUS_AUTHOR>,<US_AUTHOR>,":// www . cn bc . com / 2018 / 05 / 31 / a - 523 - month ly - payment - is - the - new - standard - for - car - buy ers . html Sorry for the formatting , on mobile . Saw this article and thought I would put this up as a P SA since there are a lot of auto loan posts on here . This is sad to see as the "" new standard .""","[0.00428769, 0.006080981, 0.0062858043, 0.010568541, 0.012739029, 0.007376492, 0.006420173, 0.004382716, 0.011183444, 0.0038852477, 0.012358462, 0.005333837, 0.01289492, 0.0041604335, 0.0046340297, 0.004655211, 0.07038398, 0.0050136596, 0.009729752, 0.007269001, 0.004854194, 0.013602063, 0.004515078, 0.0045108176, 0.0047031436, 0.008558643, 0.00589615, 0.0076290667, 0.005019474, 0.0152115505, 0.006515356, 0.007503664, 0.0062547526, 0.008839439, 0.0072120884, 0.0076134577, 0.010875404, 0.010158939, 0.0073198806, 0.006497

group=relative_time_bin


,group_1,group_2,source_text,input_attention_1,input_attention_2,attn_div
389,<RESPONSE_TIME_0_AUTHOR>,<RESPONSE_TIME_1_AUTHOR>,"âĢ Ļ ve always been of the opinion that too much screen time is one of the worst things for kids . My daughter is 12 and has wanted one forever , but I âĢ Ļ ve held firm on it by not getting her one . I âĢ Ļ m divorced . My ex and I share custody . He had her last week and got her an iPhone and put it on his plan . I âĢ Ļ m pretty convinced that this is all just a passive aggressive jab at me because he knew I would either take it away or let him be the hero who finally got her a phone . Well I stood firm in my beliefs and took it from her when she came home . Of course she âĢ Ļ s furious with me . Thanks ex . She âĢ Ļ s barely talking to me . Her point is that she hasn âĢ Ļ t even done anything and that it âĢ Ļ s not mine to take . Her dad gave it to her .","[0.0018020831, 0.0019831958, 0.0019372591, 0.0022346156, 0.0021671704, 0.0028278069, 0.0025812804, 0.0033197477, 0.0018916442, 0.0019713177, 0.0025929117, 0.0045416513, 0.0022583026, 0.002360688, 0.0018652957, 0.0035917582, 0.0029598437, 0.0034627586, 0.0018832997, 0.0032464382, 0.0018871822, 0.0038199173, 0.0017955387, 0.0025871024, 0.0033017052, 0.0022359, 0.0022985083, 0.002566436, 0.0023558727, 0.0031855595, 0.0021541761, 0.0053359084, 0.0051997416, 0.0024114256, 0.0023401969, 0.0030950438, 0.002871733, 0.0033688536, 0.002882638, 0.004511946, 0.0039604492, 0.0034785313, 0.0044575036, 0.002314319, 0.0034404574, 0.0034393102, 0.0023632436, 0.0041865846, 0.0021893517, 0.0022121, 0.0022531492, 0.0019193193, 0.0018103941, 0.0051834453, 0.004455968, 0.0015316491, 0.0024865903, 0.004241549, 0.0028890758, 0.0018000876, 0.0025365066, 0.0042119334, 0.0036217927, 0.0019185019, 0.0022228425, 0.0030611062, 0.0021374782, 0.003701155, 0.0028044113, 0.0027138167, 0.0025689667, 0.0022613946, 0....","[0.0018085131, 0.002013834, 0.0019455272, 0.0022431128, 0.0021760915, 0.00284065, 0.0025904542, 0.0033481217, 0.0018995616, 0.0019771312, 0.0026117768, 0.004569834, 0.0022682936, 0.0023672017, 0.0018734599, 0.0036091767, 0.002970576, 0.0034919798, 0.0018937472, 0.0032598916, 0.0018984139, 0.0038386048, 0.0017987462, 0.0025956854, 0.0033273227, 0.0022466546, 0.002315713, 0.002596592, 0.0023709955, 0.003197002, 0.0021631774, 0.005378301, 0.005214147, 0.002421025, 0.0023477816, 0.003101639, 0.0028814368, 0.0033761314, 0.0028980544, 0.0045481166, 0.003971421, 0.003482947, 0.00447319, 0.0023175715, 0.0033608265, 0.0034383456, 0.0023689917, 0.004196982, 0.0021911499, 0.0022168884, 0.002257776, 0.0019243837, 0.0018157432, 0.0052478136, 0.0044722385, 0.0015335423, 0.0024957748, 0.0042564236, 0.0028988244, 0.001805095, 0.002550111, 0.0042148833, 0.0036356123, 0.0019200984, 0.0022314563, 0.0030654953, 0.0021422291, 0.0037146704, 0.0028172936, 0.0027155546, 0.0025859457, 0.0022668815, 0.00271...",0.002417
620,<RESPONSE_TIME_1_AUTHOR>,<RESPONSE_TIME_0_AUTHOR>,"am having an issue with my probation officer . I had no luck going to this person 's superior for recomp ense . I have been given a deadline that I cant possibly meet . The deadline was to an alternative sanction in lieu of being brought up for a technical violation . The violation is for breaking a rule of probation - The vern acular used was you either accept the alternative sanction or go to jail . In the moment , I signed the paperwork agreeing to the alternative sanction despite voicing my objections as to the timeline . Prob ation is un sy mp athetic with respect to the concerns I have raised about the timeline they have given me . Because I am in a position where I am going to be unable to compete the task within the time frame demanded - it was explained to me that if I fail to do they will proceed with the violation hearing and bring me in front of a judge to violate me . I don 't really understand the process here . I would also rather not wait around for ...","[0.0015057422, 0.0020889142, 0.0016828661, 0.001

In [ ]:
## TODO: add questions from G1, G2; add subreddit

In [8]:
## TODO: visualize attention in blocks
class CharVal(object):
    def __init__(self, char, val):
        self.char = char
        self.val = val

    def __str__(self):
        return self.char

def rgb_to_hex(rgb):
    return '#%02x%02x%02x' % rgb
MAX_COLOR_VAL=255
# MAX_COLOR_VAL=220 # reduce intensity?
def color_charvals(s, cmap):
    # convert value to color via cmap
    # then map from RGB to hex
#     r = 255-int(s.val*255)
#     color = rgb_to_hex((255, r, r))
    cmap_color = (np.array(cmap(s.val)[:3])*MAX_COLOR_VAL).astype(int)
    cmap_color = tuple(cmap_color)
    color = rgb_to_hex(cmap_color)
    return 'background-color: %s' % color

In [9]:
import matplotlib.pyplot as plt
from matplotlib import cm
cmap = plt.get_cmap('Reds')
tokenized_text = 'this is some tokenized text'.split(' ')
attention_output = np.random.randn(len(tokenized_text))
char_vals = [CharVal(c, v) for c, v in zip(tokenized_text, attention_output)]
import pandas as pd
char_df = pd.DataFrame(char_vals).transpose()
# apply coloring values
## TODO: convert to RGB w/ custom color map in matplotlib
char_df = char_df.style.applymap(lambda x: color_charvals(x, cmap))
display(char_df)

,0,1,2,3,4
0,this,is,some,tokenized,text


In [10]:
## do the same thing for paired text
from sklearn.preprocessing import StandardScaler
def visualize_attention(data, cmap, attn_vars=['input_attention_1', 'input_attention_2'], rescale=False, rescale_offset=0.5):
    tokens = data.loc['input_tokens']
#     attn_1 = data.loc['input_attention_1']
#     attn_2 = data.loc['input_attention_2']
#     token_attn_vals_1 = list(map(lambda x: CharVal(*x), zip(tokens, attn_1)))
#     token_attn_vals_2 = list(map(lambda x: CharVal(*x), zip(tokens, attn_2)))
    # optional rescale by min/max
    if(rescale):
        for attn_var_i in attn_vars:
            data.loc[attn_var_i] = (data.loc[attn_var_i] - data.loc[attn_var_i].mean()) / data.loc[attn_var_i].std() + rescale_offset
#             data.loc[attn_var_i] = data.loc[attn_var_i] * rescale_offset
            # min-max scaling distorts min/max scores, since they tend to be close to 1.
#             min_i = data.loc[attn_var_i].min()
#             max_i = data.loc[attn_var_i].max()
#             data.loc[attn_var_i] = (data.loc[attn_var_i] - min_i) / (max_i - min_i)
            # tmp debug
#             print(f'rescaled attn = {data.loc[attn_var_i]}')
    char_df = pd.DataFrame([
        list(map(lambda x: CharVal(*x), zip(tokens, data.loc[attn_var_i])))
        for attn_var_i in attn_vars
    ]).transpose()
    char_df = char_df.style.applymap(lambda x: color_charvals(x, cmap))
    display(char_df)
    # tmp debug
#     display(data.loc[['input_attention_1', 'input_attention_2']])

In [11]:
cmap = plt.get_cmap('Reds')
for group_i, data_i in attn_div_data.groupby('reader_group'):
    data_i.sort_values('attn_div', ascending=False, inplace=True)
    print(f'group category={group_i}')
    viz_data_i = data_i.iloc[0, :]
    visualize_attention(viz_data_i, cmap, rescale=False)

group category=expert_pct_bin


,0,1
0,I,I
1,can,can
2,'t,'t
3,deal,deal
4,with,with
5,my,my
6,parents,parents
7,anymore,anymore
8,but,but
9,unfortunately,unfortunately


group category=location_region


,0,1
0,found,found
1,out,out
2,I,I
3,was,was
4,pregnant,pregnant
5,with,with
6,my,my
7,daughter,daughter
8,at,at
9,19,19


group category=relative_time_bin


,0,1
0,âĢ,âĢ
1,Ļ,Ļ
2,ve,ve
3,always,always
4,been,been
5,of,of
6,the,the
7,opinion,opinion
8,that,that
9,too,too


Let's look at the ratios instead of the raw scores.

In [12]:
attn_div_data = attn_div_data.assign(**{
    'attn_ratio_1_2' : attn_div_data.apply(lambda x: np.array(x.loc['input_attention_1']) / np.array(x.loc['input_attention_2']), axis=1)
})

In [13]:
cmap = plt.get_cmap('coolwarm')
rescale_offset = 0.5 # center distribution around 1.0 - 0.5 = 0.5
sample_per_group = 5
for group_i, data_i in attn_div_data.groupby('reader_group'):
    data_i.sort_values('attn_div', ascending=False, inplace=True)
    ## TODO: show questions after attn distribution
    for idx_j, viz_data_j in data_i.iloc[:sample_per_group, :].iterrows():
        # show raw ratio val
        print(f'id={viz_data_j.loc["article_id"]}; group category={group_i}; group_1={data_i.loc[:, "group_1"].iloc[0]} (red); group_2={data_i.loc[:, "group_2"].iloc[0]} (blue)')
#         print(viz_data_i.loc['attn_ratio_1_2'])
        visualize_attention(viz_data_j, cmap, attn_vars=['attn_ratio_1_2'], rescale=True, rescale_offset=rescale_offset)

id=bu69wd; group category=expert_pct_bin; group_1=<EXPERT_PCT_0_AUTHOR> (red); group_2=<EXPERT_PCT_1_AUTHOR> (blue)


,0
0,I
1,can
2,'t
3,deal
4,with
5,my
6,parents
7,anymore
8,but
9,unfortunately


id=9qrjlh; group category=expert_pct_bin; group_1=<EXPERT_PCT_0_AUTHOR> (red); group_2=<EXPERT_PCT_1_AUTHOR> (blue)


,0
0,wife
1,had
2,an
3,old
4,401
5,k
6,with
7,about
8,$
9,15


id=c97ggc; group category=expert_pct_bin; group_1=<EXPERT_PCT_0_AUTHOR> (red); group_2=<EXPERT_PCT_1_AUTHOR> (blue)


...

,0
0,know
1,this
2,has
3,probably
4,been
5,asked
6,a
7,million
8,times
9,","


id=barg6e; group category=relative_time_bin; group_1=<RESPONSE_TIME_0_AUTHOR> (red); group_2=<RESPONSE_TIME_1_AUTHOR> (blue)


,0
0,my
1,15
2,-
3,year
4,-
5,old
6,daughter
7,wanted
8,to
9,go


id=bt5abc; group category=relative_time_bin; group_1=<RESPONSE_TIME_0_AUTHOR> (red); group_2=<RESPONSE_TIME_1_AUTHOR> (blue)


,0
0,am
1,a
2,professor
3,that
4,lectures
5,undergraduate
6,courses
7,in
8,statistics
9,and


These results are interesting and could point to patterns:
- `NONUS` author has higher attention for family-related words (`grandmother`)
- `NOVICE` author has higher attention on specific details (`getting`, `custody`) and controversial (?) phrases (`I would either take it away`)
- `LONG-RESPONSE` author has higher attention on personal details (`convinced`, `divorced`)

In [39]:
## generate plots for paper lol
# https://github.com/jiesutd/Text-Attention-Heatmap-Visualization/blob/master/text_attention.py
def generate_attn_latex(text_list, attention_list, positive_color='red', negative_color='blue', rescale_value = 50):
    assert(len(text_list) == len(attention_list))
    pos_idx = np.where(attention_list >= 0.)[0]
    neg_idx = np.where(attention_list < 0.)[0]
    if(rescale_value > 1.):
        # norm scale
#         rescale_attention_list = abs((attention_list - np.mean(attention_list)) / np.std(attention_list))
        # min-max scale
        # get min/max over all data
        abs_attention_list = abs(attention_list)
        attention_max = max(abs_attention_list)
        attention_min = min(abs_attention_list)
        rescale_attention_list = (abs_attention_list - attention_min) / (attention_max - attention_min)
        rescale_attention_list = rescale_attention_list * rescale_value
#         rescale_attention_list[neg_idx] = rescale_attention_list[neg_idx]
        attention_list = rescale_attention_list
#         print(pos_idx, neg_idx)
#         print(attention_list[pos_idx])
#         attention_list[pos_idx] = rescale(attention_list[pos_idx])
#         print(attention_list[neg_idx])
#         attention_list[neg_idx] = rescale(-1*attention_list[neg_idx])
    word_num = len(text_list)
    text_list = clean_word(text_list)
#     \usepackage{color}
# \usepackage{tcolorbox}
# \usepackage{CJK}
# \usepackage{adjustbox}
# \tcbset{width=0.9\textwidth,boxrule=0pt,colback=red,arc=0pt,auto outer arc,left=0pt,right=0pt,boxsep=5pt}
    latex_str = ''
#     latex_str += r'\begin{CJK*}{UTF8}{gbsn}'
#     latex_str += r'''{\setlength{\fboxsep}{0pt}\colorbox{white!0}{\parbox{0.9\textwidth}{'''+"\n"
    for idx in range(word_num):
        if(idx in pos_idx):
            color = positive_color
        elif(idx in neg_idx):
            color = negative_color
        latex_str += "\\colorbox{%s!%.2f}{"%(color, attention_list[idx])+"\\strut " + text_list[idx]+"} "
#     latex_str += "\n}}}"
#     latex_str += '\n'
#     latex_str += r'''\end{CJK*}'''
    return latex_str

def clean_word(word_list):
    new_word_list = []
    for word in word_list:
        for latex_sensitive in ["\\", "%", "&", "^", "#", "_", "$",  "{", "}"]:
            if latex_sensitive in word:
                word = word.replace(latex_sensitive, '\\'+latex_sensitive)
        new_word_list.append(word)
    return new_word_list

In [16]:
test_words = 'this is a test sentence about some words and some more words'.split(' ')
test_attn = np.random.randn(len(test_words))
print(test_attn)
attn_latex_str = generate_attn_latex(test_words, test_attn, rescale_value=50.)
print(attn_latex_str)

[-0.05359755  0.27311723  0.41570891  0.68750255 -0.44343196 -0.19781917
 -0.10847102 -1.17618409 -0.79379699 -0.30696218  0.96522094  2.01285865]
\colorbox{blue!0.00}{\strut this} \colorbox{red!5.60}{\strut is} \colorbox{red!9.24}{\strut a} \colorbox{red!16.18}{\strut test} \colorbox{blue!9.95}{\strut sentence} \colorbox{blue!3.68}{\strut about} \colorbox{blue!1.40}{\strut some} \colorbox{blue!28.65}{\strut words} \colorbox{blue!18.89}{\strut and} \colorbox{blue!6.47}{\strut some} \colorbox{red!23.26}{\strut more} \colorbox{red!50.00}{\strut words} 


Let's get snippets for each of the examples above and convert them to latex.

In [47]:
pd.set_option('display.max_colwidth', 5000)
latex_text_len = 50
sample_per_group = 10
latex_color_text_data = []
rescale_value = 50.
min_neg_attn_count = 10
min_pos_attn_count = 10
for group_i, data_i in attn_div_data.groupby('reader_group'):
    data_i.sort_values('attn_div', ascending=False, inplace=True)
    group_1 = data_i.loc[:, 'group_1'].iloc[0]
    group_2 = data_i.loc[:, 'group_2'].iloc[0]
    # limit to questions w/ visible variation
    data_i = data_i[data_i.loc[:, 'attn_ratio_1_2'].apply(lambda x: (x[:latex_text_len]>1.).sum() >= min_pos_attn_count and (x[:latex_text_len]<1.).sum() >= min_neg_attn_count)]
    for idx_j, viz_data_j in data_i.iloc[:sample_per_group, :].iterrows():
        # show raw ratio val
#         print(f'id={viz_data_j.loc["article_id"]}; group category={group_i}; group_1={group_1} (red); group_2={group_2} (blue)')
#         print(viz_data_i.loc['attn_ratio_1_2'])
#         visualize_attention(viz_data_j, cmap, attn_vars=['attn_ratio_1_2'], rescale=True, rescale_offset=rescale_offset)
        attn_j = viz_data_j.loc['attn_ratio_1_2']-1
        tokens_j = viz_data_j.loc['input_tokens']
        attn_j = attn_j[:latex_text_len]
        tokens_j = tokens_j[:latex_text_len]
        attn_color_j = generate_attn_latex(tokens_j, attn_j, rescale_value=rescale_value)
        latex_color_text_data.append([viz_data_j.loc['article_id'], group_i, group_1, group_2, attn_color_j])
latex_color_text_data = pd.DataFrame(latex_color_text_data, columns=['article_id', 'reader_group', 'group_1', 'group_2', 'attn_color_tex'])
display(latex_color_text_data.loc[:, ['reader_group', 'group_1', 'group_2', 'attn_color_tex']])

,reader_group,group_1,group_2,attn_color_tex
0,expert_pct_bin,<EXPERT_PCT_0_AUTHOR>,<EXPERT_PCT_1_AUTHOR>,\colorbox{red!3.15}{\strut added} \colorbox{red!0.32}{\strut 8} \colorbox{red!2.48}{\strut GB} \colorbox{blue!1.66}{\strut more} \colorbox{blue!8.42}{\strut RAM} \colorbox{red!2.72}{\strut and} \colorbox{red!1.84}{\strut i} \colorbox{red!0.86}{\strut cant} \colorbox{blue!2.98}{\strut get} \colorbox{red!1.59}{\strut it} \colorbox{red!1.24}{\strut to} \colorbox{red!0.92}{\strut run} \colorbox{blue!1.60}{\strut at} \colorbox{blue!7.00}{\strut 3000} \colorbox{red!7.03}{\strut MHz} \colorbox{red!1.25}{\strut anymore} \colorbox{red!4.25}{\strut .} \colorbox{red!2.58}{\strut it} \colorbox{red!0.43}{\strut just} \colorbox{red!2.63}{\strut causes} \colorbox{red!2.59}{\strut my} \colorbox{blue!0.23}{\strut pc} \colorbox{red!1.15}{\strut to} \colorbox{blue!0.30}{\strut boot} \colorbox{red!0.97}{\strut up} \colorbox{red!1.17}{\strut in} \colorbox{red!1.92}{\strut safe} \colorbox{red!2.26}{\strut mode} \colorbox{red!3.41}{\strut until} \colorbox{red!0.99}{\strut i} \colorbox{blue!2.34}{\strut reduce} \colorbox{red!0.77}{\strut the} \colorbox{blue!0.00}{\strut RAM} \colorbox{red!2.14}{\strut speed} \colorbox{blue!0.03}{\strut to} \colorbox{red!2.60}{\strut 2400} \colorbox{red!7.08}{\strut MHz} \colorbox{red!1.79}{\strut and} \colorbox{blue!1.54}{\strut only} \colorbox{blue!4.50}{\strut then} \colorbox{red!0.28}{\strut will} \colorbox{red!0.92}{\strut it} \colorbox{blue!0.32}{\strut boot} \colorbox{red!1.72}{\strut .} \colorbox{red!0.08}{\strut Any} \colorbox{red!1.91}{\strut ideas} \colorbox{blue!2.21}{\strut ?} \colorbox{blue!50.00}{\strut <EXPERT\_PCT\_1\_AUTHOR>}
1,expert_pct_bin,<EXPERT_PCT_0_AUTHOR>,<EXPERT_PCT_1_AUTHOR>,"\colorbox{red!1.66}{\strut ,} \colorbox{red!5.96}{\strut I} \colorbox{blue!0.43}{\strut have} \colorbox{red!6.60}{\strut studied} \colorbox{red!0.48}{\strut therm} \colorbox{red!50.00}{\strut odynamics} \colorbox{red!2.65}{\strut and} \colorbox{red!10.20}{\strut I} \colorbox{blue!0.00}{\strut wonder} \colorbox{blue!1.54}{\strut why} \colorbox{red!2.27}{\strut there} \colorbox{red!0.04}{\strut is} \colorbox{red!0.65}{\strut so} \colorbox{red!0.23}{\strut much} \colorbox{red!45.80}{\strut nons} \colorbox{red!3.01}{\strut ens} \colorbox{red!1.87}{\strut connection} \colorbox{blue!4.22}{\strut at} \colorbox{blue!0.23}{\strut water} \colorbox{red!1.49}{\strut cool} \colorbox{red!2.17}{\strut ed} \colorbox{blue!0.19}{\strut setup} \colorbox{red!5.69}{\strut .} \colorbox{red!0.51}{\strut P} \colorbox{red!15.66}{\strut ipes} \colorbox{blue!10.36}{\strut going} \colorbox{blue!8.20}{\strut do} \colorbox{red!5.69}{\strut and} \colorbox{blue!0.37}{\strut up} \colorbox{red!1.33}{\strut -} \colorbox{blue!1.12}{\strut like} \colorbox{red!0.66}{\strut in} \colorbox{red!1.97}{\strut this} \colorbox{red!1.79}{\strut setup} \colorbox{red!3.18}{\strut .} \colorbox{blue!0.41}{\strut Is} \colorbox{red!1.15}{\strut there} \colorbox{red!23.94}{\strut pumps} \colorbox{red!0.58}{\strut in} \colorbox{red!0.43}{\strut these} \colorbox{red!7.03}{\strut setups} \colorbox{red!0.01}{\strut or} \colorbox{blue!4.75}{\strut is} \colorbox{blue!7.77}{\strut natural} \colorbox{red!40.67}{\strut circulation} \colorbox{red!5.08}{\strut driven} \colorbox{blue!0.41}{\strut ?} \colorbox{blue!5.50}{\strut https} \colorbox{red!0.00}{\strut ://} \colorbox{red!3.09}{\strut www}"
2,expert_pct_bin,<EXPERT_PCT_0_AUTHOR>,<EXPERT_PCT_1_AUTHOR>,\colorbox{blue!0.51}{\strut step} \colorbox{blue!2.04}{\strut father} \colorbox{blue!0.47}{\strut âĢ} \colorbox{red!0.68}{\strut Ļ} \colorbox{red!0.00}{\strut s} \colorbox{blue!1.02}{\strut lawyer} \colorbox{blue!1.06}{\strut said} \colorbox{blue!6.38}{\strut he} \colorbox{blue!0.15}{\strut isn} \colorbox{blue!0.61}{\strut âĢ} \colorbox{blue!0.51}{\strut Ļ} \colorbox{blue!7.35}{\strut t} \colorbox{blue!0.74}{\strut allowed} \colorbox{blue!0.61}{\strut to} \colorbox{blue!0.34}{\strut have} \colorbox{red!0.13}{\strut a} \colorb

It's hard to know exactly what the model is doing under the hood, but if we had the generated questions then we might be able to see how the attention distributions contribute to the final predictions.

In [ ]:
## this is too clunky for the paper, let's just convert directly to TeX and visualize